In [1]:
import os
import cv2
import random
import numpy as np
import openslide
import matplotlib.pyplot as plt
from skimage.color import rgb2hsv
from skimage.filters import threshold_otsu
from scipy import ndimage

from Aslide import aslide

In [2]:
# def maxpooling(img, kernel_size=11):
#     h, w = img.shape
#     img_out = np.ones((h, w))
#     window_size = kernel_size // 2
#     for j in range(window_size*5, h-window_size*5):
#         for i in range(window_size*5, w-window_size*5):
#             img_out[j][i] = np.max(img[j-window_size:j+window_size, i-window_size:i+window_size])
#     return img_out.astype(np.uint8)

# def maxpooling(img, kernel_size=(5,5)):
#     h, w = img.shape
#     img_out = np.ones((h, w))
#     h_win_size, w_win_size = kernel_size[0] // 2, kernel_size[1] // 2
#     margin = 10
#     for j in range(margin, h-margin):
#         for i in range(margin, w-margin):
#             img_out[j][i] = np.max(img[j-h_win_size:j+h_win_size, i-w_win_size:i+w_win_size])
#     return img_out.astype(np.uint8)

def maxpooling(img, kernel_size=(5,5)):
    h, w = img.shape
    img_out = np.ones((h, w))
    h_win_size, w_win_size = kernel_size
    margin = 10
    for j in range(margin, h//2-margin):
        for i in range(margin, w//2-margin):
            img_out[j][i] = np.max(img[j:j+h_win_size, i:i+w_win_size])
    for j in range(h//2-margin, h-margin):
        for i in range(margin, w//2-margin):
            img_out[j][i] = np.max(img[j-h_win_size:j, i:i+w_win_size])
    for j in range(margin, h//2-margin):
        for i in range(w//2-margin, w-margin):
            img_out[j][i] = np.max(img[j:j+h_win_size, i-w_win_size:i])
    for j in range(h//2-margin, h-margin):
        for i in range(w//2-margin, w-margin):
            img_out[j][i] = np.max(img[j-h_win_size:j, i-w_win_size:i])
    return img_out.astype(np.uint8)

In [3]:
def find_roi(wsi_path, save_path):
    try:
        slide = openslide.OpenSlide(wsi_path)
    except:
        slide = aslide.Aslide(wsi_path)
    level = slide.level_count - 1
    downsample = slide.level_downsamples[level]
    m, n = slide.dimensions

    source = np.array(slide.read_region((0, 0), level, slide.level_dimensions[level]).convert('RGB'))
    slide.close()
    img_gray = cv2.cvtColor(source, cv2.COLOR_RGB2GRAY)
    img_gray = cv2.pyrDown(img_gray)
    img_gray = cv2.pyrDown(img_gray)
    
#     kernel_size = (3, 3)
#     kernel = np.ones(kernel_size, np.uint8)
#     img_open = cv2.morphologyEx(img_gray, cv2.MORPH_ERODE, kernel, iterations=5)
#     img_open = cv2.morphologyEx(img_open, cv2.MORPH_DILATE, kernel, iterations=3)

    kernel_size = (5, 5)
    kernel = np.ones(kernel_size, np.uint8)
    img_open = cv2.morphologyEx(img_gray, cv2.MORPH_OPEN, kernel, iterations=3)
    # img_open = cv2.morphologyEx(img_open, cv2.MORPH_CLOSE, kernel, iterations=1)
    # img_open = cv2.morphologyEx(img_open, cv2.MORPH_ERODE, kernel, iterations=1)

    tissue_mask = img_open > threshold_otsu(img_open) / 0.99
    img_out = maxpooling(tissue_mask, kernel_size=(1, 5))
    img_out = maxpooling(img_out, kernel_size=(5, 1))

    img, contours, hierarchy = cv2.findContours(img_out.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
#     print(len(contours))

    hull = cv2.convexHull(contours[1], False) * 4
#     cv2.drawContours(source, [hull], 0, color=(random.randint(0,255),random.randint(0,255),random.randint(0,255)), thickness=5)
# #     plt.imshow(source)
#     save_name = os.path.join(save_path, os.path.basename(wsi_path) + ".jpg")
#     cv2.imwrite(save_name, source)
    
    area = cv2.contourArea(hull) * downsample * downsample
    area_ratio = area / m / n
    print(wsi_path, area_ratio)

In [4]:
data_path = "/home/nvme/batch6.4-added/WSIS-added"
wsi_paths = [os.path.join(data_path, f) for f in os.listdir(data_path) if f.endswith('.tif') or f.endswith('.kfb')]
save_path = "/home/nvme/batch6.4-added/WSIS-added-contours-noequalize-unilateral-maxpooling5-1"
os.makedirs(save_path, exist_ok=True)

# wsi_path = "/home/nvme/batch6.4-added/WSIS-added/SZH00904.TMAP"
for wsi_path in wsi_paths:
    find_roi(wsi_path, save_path)

/home/nvme/batch6.4-added/WSIS-added/TC18000963.kfb 0.7066267806267805
/home/nvme/batch6.4-added/WSIS-added/BD1601770.kfb 0.6986164705882354
/home/nvme/batch6.4-added/WSIS-added/2018-05-08-16_52_28.tif 0.6560320248114279
/home/nvme/batch6.4-added/WSIS-added/L16-04738.kfb 0.7965928571428571
/home/nvme/batch6.4-added/WSIS-added/1715428.kfb 0.7135333333333334
/home/nvme/batch6.4-added/WSIS-added/BD1608489.kfb 0.6645188536953243
/home/nvme/batch6.4-added/WSIS-added/L16-14450.kfb 0.8013296703296704
/home/nvme/batch6.4-added/WSIS-added/L15-22188.kfb 0.7828480752277402
/home/nvme/batch6.4-added/WSIS-added/BD1513891.kfb 0.7203044871794871
/home/nvme/batch6.4-added/WSIS-added/BD1600249.kfb 0.6953286274509803
/home/nvme/batch6.4-added/WSIS-added/TC18007248.kfb 0.7255388291517323
/home/nvme/batch6.4-added/WSIS-added/L16-13957.kfb 0.5224905426356589
/home/nvme/batch6.4-added/WSIS-added/BD1508607.kfb 0.7063476923076923
/home/nvme/batch6.4-added/WSIS-added/TB18008128.kfb 0.6806812104152006
/home/nvm

/home/nvme/batch6.4-added/WSIS-added/2017-11-24-12_51_13.tif 0.6522000013537457
/home/nvme/batch6.4-added/WSIS-added/2017-11-24-13_37_27.tif 0.6162416320199557
/home/nvme/batch6.4-added/WSIS-added/L16-11805.kfb 0.8115788823711022
/home/nvme/batch6.4-added/WSIS-added/TC18008831.kfb 0.7000793783217876
/home/nvme/batch6.4-added/WSIS-added/BD1502454.kfb 0.6349706293706293
/home/nvme/batch6.4-added/WSIS-added/BD1502748.kfb 0.75181
/home/nvme/batch6.4-added/WSIS-added/BD1505187.kfb 0.6998595918367347
/home/nvme/batch6.4-added/WSIS-added/TC17011016.kfb 0.6012132186901761
/home/nvme/batch6.4-added/WSIS-added/BD02543.kfb 0.7283
/home/nvme/batch6.4-added/WSIS-added/BD1605248.kfb 0.6294582425562818
/home/nvme/batch6.4-added/WSIS-added/TC18032834.kfb 0.7582089947089947
/home/nvme/batch6.4-added/WSIS-added/TB18003837.kfb 0.8133053278688525
/home/nvme/batch6.4-added/WSIS-added/TC17015379.kfb 0.6806451435118307
/home/nvme/batch6.4-added/WSIS-added/TC17058959.kfb 0.6193256350856643
/home/nvme/batch6.4

/home/nvme/batch6.4-added/WSIS-added/L16-15776.kfb 0.8737003861003861
/home/nvme/batch6.4-added/WSIS-added/TC18033968.kfb 0.7848118518518519
/home/nvme/batch6.4-added/WSIS-added/L16-04377.kfb 0.8309513157894737
/home/nvme/batch6.4-added/WSIS-added/2018-05-08-17_52_07.tif 0.6736572146812799
/home/nvme/batch6.4-added/WSIS-added/TC17023813.kfb 0.7161767578577242
/home/nvme/batch6.4-added/WSIS-added/TC17039583.kfb 0.6671997970087337
/home/nvme/batch6.4-added/WSIS-added/L18-18665.kfb 0.7722722631877482
/home/nvme/batch6.4-added/WSIS-added/TB18003651.kfb 0.6415597057159027
/home/nvme/batch6.4-added/WSIS-added/TC18021919.kfb 0.6895690777404279
/home/nvme/batch6.4-added/WSIS-added/2017-11-24-13_16_54.tif 0.5615276517616122
/home/nvme/batch6.4-added/WSIS-added/TC18001238.kfb 0.6577931034482759
/home/nvme/batch6.4-added/WSIS-added/2018-01-15-15_25_37.tif 0.23221425258054648
/home/nvme/batch6.4-added/WSIS-added/2017-11-24-13_19_06.tif 0.6668670797529486
/home/nvme/batch6.4-added/WSIS-added/L18-10

/home/nvme/batch6.4-added/WSIS-added/2017-11-24-13_46_24.tif 0.6895962494337091
/home/nvme/batch6.4-added/WSIS-added/TC17018659.kfb 0.6182875125766958
/home/nvme/batch6.4-added/WSIS-added/SZH1611418.kfb 0.5863485086342229
/home/nvme/batch6.4-added/WSIS-added/2017-11-24-13_12_52.tif 0.6079052732003911
/home/nvme/batch6.4-added/WSIS-added/BD1504901.kfb 0.7014976
/home/nvme/batch6.4-added/WSIS-added/TC18009269.kfb 0.004962406015037594
/home/nvme/batch6.4-added/WSIS-added/TC17053680.kfb 0.7510254370054975
/home/nvme/batch6.4-added/WSIS-added/L16-15660.kfb 0.8599083333333334
/home/nvme/batch6.4-added/WSIS-added/TC17011618.kfb 0.6660247340955648
/home/nvme/batch6.4-added/WSIS-added/L18-18425.kfb 0.7887249007373794
/home/nvme/batch6.4-added/WSIS-added/1700683.kfb 0.7348566666666666
/home/nvme/batch6.4-added/WSIS-added/L16-16198.kfb 0.8026746666666666
/home/nvme/batch6.4-added/WSIS-added/TC18006133.kfb 0.69336807582996
/home/nvme/batch6.4-added/WSIS-added/TC17018994.kfb 0.10771085771788316
/ho

/home/nvme/batch6.4-added/WSIS-added/XB1806204.kfb 0.6644928774928774
/home/nvme/batch6.4-added/WSIS-added/BD1501723.kfb 0.637761689291101
/home/nvme/batch6.4-added/WSIS-added/TC18003448.kfb 0.3062559906487434
/home/nvme/batch6.4-added/WSIS-added/TC18008679.kfb 0.7482178103291866
/home/nvme/batch6.4-added/WSIS-added/TC17021756.kfb 0.6702531113419271
/home/nvme/batch6.4-added/WSIS-added/TC18012177.kfb 0.6955023195864024
/home/nvme/batch6.4-added/WSIS-added/2017-11-24-14_05_33.tif 0.6292674821447556
/home/nvme/batch6.4-added/WSIS-added/2017-11-24-12_18_52.tif 0.6311737377562289
/home/nvme/batch6.4-added/WSIS-added/TC17062557.kfb 0.6632077350427619
/home/nvme/batch6.4-added/WSIS-added/TC18029937.kfb 0.6614097245641253
/home/nvme/batch6.4-added/WSIS-added/2018-05-08-16_21_58.tif 0.2617598925131397
/home/nvme/batch6.4-added/WSIS-added/TC18034210.kfb 0.6305888119895484
/home/nvme/batch6.4-added/WSIS-added/TC18002902.kfb 0.6445457641783722
/home/nvme/batch6.4-added/WSIS-added/TC18007459.kfb 0

/home/nvme/batch6.4-added/WSIS-added/L18-16820.kfb 0.8476588940706196
/home/nvme/batch6.4-added/WSIS-added/TC17010665.kfb 0.6478682475607277
/home/nvme/batch6.4-added/WSIS-added/TC17063213.kfb 0.7295580642682428
/home/nvme/batch6.4-added/WSIS-added/TC18024250.kfb 0.73492435985526
/home/nvme/batch6.4-added/WSIS-added/L16-11795.kfb 0.7917328385899814
/home/nvme/batch6.4-added/WSIS-added/2018-01-15-15_13_04.tif 0.6409569323309439
/home/nvme/batch6.4-added/WSIS-added/2018-05-08-16_26_21.tif 0.6711198906709415
/home/nvme/batch6.4-added/WSIS-added/TC18011839.kfb 0.679207852068985
/home/nvme/batch6.4-added/WSIS-added/2017-11-24-14_25_01.tif 0.6327877094391511
/home/nvme/batch6.4-added/WSIS-added/L18-18602.kfb 0.7395270562770563
/home/nvme/batch6.4-added/WSIS-added/XB1810303.kfb 0.637478021978022
/home/nvme/batch6.4-added/WSIS-added/NJ18088302.kfb 0.5560047829613823
/home/nvme/batch6.4-added/WSIS-added/XB1803929.kfb 0.6267196263678697
/home/nvme/batch6.4-added/WSIS-added/SZH1609689.kfb 0.58673